In [9]:
import ee 
import os 
import geemap
import geopandas as gpd 
import numpy as np
import time
from glob import glob

def tileindex(tdir,gpkg): #pkg = tdir + '.gpkg'
    cmd = f'gdaltindex -t_srs EPSG:4326 -f GPKG {gpkg} {tdir}/*.tif '
    os.system(cmd)

def geotile_generate_tiles(tif_path, gpkg_path, dir_path):
    from geotile import GeoTile
    
    if not os.path.isfile(gpkg_path):
        gt = GeoTile(tif_path)
        gt.generate_tiles(dir_path,tile_x=256, tile_y=256, stride_x=256, stride_y=256)
        gt.close()

        #gpkg = dir_path + '.gpkg'
        #gpkg = os.path.join(dir_path+'.gpkg')
        tileindex(dir_path,gpkg_path) #gpkg = tdir + '.gpkg'
    else: print(f'gpkg file already exists {gpkg_path}')

In [10]:
gpkg_out = '/media/ljp238/6tb/Joseph/DATASETS/mekong_delta_data_tiles/wdir/1x1degree/gpkg_patches'
os.makedirs(gpkg_out, exist_ok=True)

In [11]:
data_path  = '/media/ljp238/6tb/Joseph/DATASETS/mekong_delta_data_tiles/wdir/1x1degree/'
fs = glob(f'{data_path}/*/*.tif'); print(len(fs))
tdxs = [i for i in fs if 'TANDEMX.tif' in i]
gpkgs = [os.path.join(gpkg_out, os.path.basename(i).replace('.tif', '.gpkg')) for i in tdxs ]
tdxs[0],gpkgs[0],

95


('/media/ljp238/6tb/Joseph/DATASETS/mekong_delta_data_tiles/wdir/1x1degree/N09_E105/N09_E105_TANDEMX.tif',
 '/media/ljp238/6tb/Joseph/DATASETS/mekong_delta_data_tiles/wdir/1x1degree/gpkg_patches/N09_E105_TANDEMX.gpkg')

In [15]:
from concurrent.futures import ProcessPoolExecutor

In [16]:
if __name__ == '__main__':
    with ProcessPoolExecutor() as PEX:
        for i in range(len(tdxs)):

            gpkg_tile = os.path.join(gpkg_out, os.path.basename(tdxs[i]).replace('.tif',''))
            
            PEX.submit(geotile_generate_tiles,tdxs[i], gpkgs[i], gpkg_tile)
            


gpkg file already exists /media/ljp238/6tb/Joseph/DATASETS/mekong_delta_data_tiles/wdir/1x1degree/gpkg_patches/N09_E106_TANDEMX.gpkg
gpkg file already exists /media/ljp238/6tb/Joseph/DATASETS/mekong_delta_data_tiles/wdir/1x1degree/gpkg_patches/N10_E105_TANDEMX.gpkggpkg file already exists /media/ljp238/6tb/Joseph/DATASETS/mekong_delta_data_tiles/wdir/1x1degree/gpkg_patches/N09_E105_TANDEMX.gpkg

gpkg file already exists /media/ljp238/6tb/Joseph/DATASETS/mekong_delta_data_tiles/wdir/1x1degree/gpkg_patches/N10_E104_TANDEMX.gpkg
gpkg file already exists /media/ljp238/6tb/Joseph/DATASETS/mekong_delta_data_tiles/wdir/1x1degree/gpkg_patches/N10_E106_TANDEMX.gpkg


In [17]:
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [18]:
os.getcwd()

'/home/ljp238/Documents/phdprojects/dem_mekong_delta/dem_mekong_delta/gee_download'

In [26]:
def ee_clip_mosaic_roi(dobject,roi):
    mosaic = dobject.mosaic()
    mosaiclip = mosaic.clip(roi)
    return mosaiclip


In [27]:
from concurrent.futures import ThreadPoolExecutor
def getDEM_files(roi):
    glo30  = ee.ImageCollection("COPERNICUS/DEM/GLO30").filterBounds(roi)
    dem = glo30.select('DEM')
    edm = glo30.select('EDM')
    flm = glo30.select('FLM')
    hem = glo30.select('HEM')
   # wbm = glo30.select('WBM')

    wbm = ee_clip_mosaic_roi(glo30.select('WBM'),roi)

    
    return dem,edm,flm,hem,wbm

def get_ee_geometry(i, g, name):
    ig = g.iloc[i:i+1,]
    bBox = [float(ig.minx), float(ig.miny), float(ig.maxx), float(ig.maxy)]
    fname = (os.path.basename(ig.location.values[0])).replace('..tif', f'_{name}.tif')
    region = ee.Geometry.Rectangle(bBox)
    return region, fname

def gee_download_geemap(image,outpath, scale):
    print(outpath)
   # image = ee.Image(image)
    if os.path.isfile(outpath):
        print('Already downloaded') 
    else:
        geemap.ee_export_image(image, outpath, scale=scale)

In [34]:
outdir = '/media/ljp238/6tb/Joseph/DATASETS/COPDEM/'
VAR = 'WBM'

In [37]:
# do 1 for each variable :: all scripts in parallel 
if __name__ == '__main__':
    with ThreadPoolExecutor(30) as PEX:
        for fi in range(len(gpkgs)):
            #if fi > 0: break
            gfile = gpkgs[fi]
            g = gpd.read_file(gfile)
            g[['minx','miny','maxx','maxy']] = g.bounds
            #print(gfile)
            for i in range(g.shape[0]):
                roi, fname = get_ee_geometry(i,g, VAR)
                _,_,_,_,wbm = getDEM_files(roi)
                tiledir_var = os.path.join(outdir,os.path.basename(gfile).replace('_TANDEMX.gpkg', ''), VAR)
                os.makedirs(tiledir_var, exist_ok=True)
                outpath = os.path.join(tiledir_var, fname)
                PEX.submit(gee_download_geemap,wbm,outpath, 12)





/media/ljp238/6tb/Joseph/DATASETS/COPDEM/N09_E105/WBM/tile_0_0_WBM.tif
Generating URL ...
/media/ljp238/6tb/Joseph/DATASETS/COPDEM/N09_E105/WBM/tile_0_1024_WBM.tif
Generating URL ...
/media/ljp238/6tb/Joseph/DATASETS/COPDEM/N09_E105/WBM/tile_0_1280_WBM.tif
Generating URL ...
/media/ljp238/6tb/Joseph/DATASETS/COPDEM/N09_E105/WBM/tile_0_1536_WBM.tif
Generating URL ...
/media/ljp238/6tb/Joseph/DATASETS/COPDEM/N09_E105/WBM/tile_0_1792_WBM.tif
Generating URL ...
/media/ljp238/6tb/Joseph/DATASETS/COPDEM/N09_E105/WBM/tile_0_2048_WBM.tif
Generating URL ...
/media/ljp238/6tb/Joseph/DATASETS/COPDEM/N09_E105/WBM/tile_0_2304_WBM.tif
Generating URL ...
/media/ljp238/6tb/Joseph/DATASETS/COPDEM/N09_E105/WBM/tile_0_256_WBM.tif
Generating URL ...
/media/ljp238/6tb/Joseph/DATASETS/COPDEM/N09_E105/WBM/tile_0_2560_WBM.tif
Generating URL ...
/media/ljp238/6tb/Joseph/DATASETS/COPDEM/N09_E105/WBM/tile_0_2816_WBM.tif
Generating URL ...
/media/ljp238/6tb/Joseph/DATASETS/COPDEM/N09_E105/WBM/tile_0_3072_WBM.tif


In [ ]:
# vrt file 
# raster file 